In [35]:
import os
import json
import pandas as pd

#### The maps' visited place data came separated in year folders, and then separated by month files. So I had to iterate over the folders and files and then concatenate them

In [51]:
list_folders = next(os.walk('.'))[1]

df_list = []

for folder in list_folders:
    files = next(os.walk(folder))[2]
    
    for file in files:
        if '.json' in file:
            with open(f'./{folder}/{file}','r', encoding='utf8') as f:  
                data = json.loads(f.read())

                temp_df = pd.json_normalize(
                    data, 
                    record_path='timelineObjects')

                df_list.append(temp_df)            

In [128]:
df = pd.concat(df_list)

I will use only the visited places data. So I will filter them.

In [129]:
place_visit_columns = df.columns[df.columns.str.contains('placeVisit')]

In [130]:
df = df[place_visit_columns]

In [131]:
df.sample(5)

,placeVisit.location.latitudeE7,placeVisit.location.longitudeE7,placeVisit.location.placeId,placeVisit.location.address,placeVisit.location.semanticType,placeVisit.location.locationConfidence,placeVisit.duration.startTimestampMs,placeVisit.duration.endTimestampMs,placeVisit.centerLatE7,placeVisit.centerLngE7,placeVisit.visitConfidence,placeVisit.otherCandidateLocations,placeVisit.editConfirmationStatus,placeVisit.location.name,placeVisit.simplifiedRawPath.points,placeVisit.childVisits,placeVisit.location.sourceInfo.deviceTag,placeVisit.placeConfidence
187,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
168,-37417533.0,-385386851.0,ChIJsevs2w9JxwcRvUMn6w0MCmc,"Av. da Universidade, 2853 - Benfica\nFortaleza...",NaN,51.199165,1556055655735,1556061821000,-37402266.0,-385392521.0,93.0,"[{'latitudeE7': -37402909, 'longitudeE7': -385...",NOT_CONFIRMED,Universidade Federal do Ceará,NaN,"[{'location': {'latitudeE7': -37414842, 'longi...",-7.103875e+08,MEDIUM_CONFIDENCE
16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
43,-37219302.0,-385195067.0,ChIJ7W_aJUlIxwcR-JRMiLIdzAc,"R. Dragão do Mar, 160 - Praia de Iracema\nFort...",NaN,97.214690,1505243474128,1505243781944,-37220413.0,-385196450.0,91.0,"[{'latitudeE7': -37213003, 'longitudeE7': -385...",NOT_CONFIRMED,Porto Iracema das Artes,NaN,NaN,-1.793314e+09,HIGH_CONFIDENCE
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [132]:
df['placeVisit.location.name'].value_counts().head()

Universidade Federal do Ceará                                        136
Shopping Benfica                                                     120
Centro de Humanidades - Área III da Universidade Federal do Ceará    117
IFCE - Federal Institute of Ceará Annex Aldeota                       76
Humanities Center 1 - UFC                                             72
Name: placeVisit.location.name, dtype: int64

I will remove my sensitive data, droping the residencial adresses.

In [135]:
for word in ['rua', 'r.', 'av.', 'avenida']:
    df = df[
        (
            df['placeVisit.location.name']
            .str.lower().str
            .startswith(word) == False
        )
    ]

In [140]:
df['placeVisit.location.name'].value_counts().head(10)

Universidade Federal do Ceará                                                      136
Shopping Benfica                                                                   120
Centro de Humanidades - Área III da Universidade Federal do Ceará                  117
IFCE - Federal Institute of Ceará Annex Aldeota                                     76
Humanities Center 1 - UFC                                                           72
CUCA Mondubim                                                                       70
Instituto Federal de Educação, Ciência e Tecnologia do Ceará | Campus Fortaleza     63
Ceará State University - Itaperi Campus                                             59
Praça da Gentilândia                                                                55
E.E.M Adauto Bezerra                                                                37
Name: placeVisit.location.name, dtype: int64

In [142]:
df.head(3)

,placeVisit.location.latitudeE7,placeVisit.location.longitudeE7,placeVisit.location.placeId,placeVisit.location.address,placeVisit.location.semanticType,placeVisit.location.locationConfidence,placeVisit.duration.startTimestampMs,placeVisit.duration.endTimestampMs,placeVisit.centerLatE7,placeVisit.centerLngE7,placeVisit.visitConfidence,placeVisit.otherCandidateLocations,placeVisit.editConfirmationStatus,placeVisit.location.name,placeVisit.simplifiedRawPath.points,placeVisit.childVisits,placeVisit.location.sourceInfo.deviceTag,placeVisit.placeConfidence
5,-38651245.0,-386326169.0,ChIJYwXCZyJTxwcRZ29RPIXsYpM,"Av. Padre José Holanda do Vale, 2071 - Centro\...",NaN,12.812797,1379959926740,1379963294968,-38639340.0,-386341140.0,76.0,"[{'latitudeE7': -38649812, 'longitudeE7': -386...",NOT_CONFIRMED,CEJAM - Centro de Educação de Jovens e Adultos...,NaN,NaN,NaN,NaN
6,-38649812.0,-386328513.0,ChIJr5zyiCJTxwcRK8J_lPB5nr4,"Avenida Pe José Holanda do Vale, 2071 - Centro...",NaN,12.898458,1379968186100,1379969773492,-38639340.0,-386341140.0,76.0,"[{'latitudeE7': -38644594, 'longitudeE7': -386...",NOT_CONFIRMED,Conselho Escolar da Escola Municipal de Ensino...,NaN,NaN,NaN,NaN
0,-38354308.0,-385539145.0,ChIJz5vRGeRRxwcRNElOsYTpcoI,Pref. José Walter\nFortaleza - CE\n60810-670\n...,NaN,30.073748,1408143929408,1408146505596,-38351582.0,-385535241.0,64.0,"[{'latitudeE7': -38331968, 'longitudeE7': -385...",NOT_CONFIRMED,Refla Construções e Serviços,NaN,NaN,NaN,NaN


In [143]:
df.columns

Index(['placeVisit.location.latitudeE7', 'placeVisit.location.longitudeE7',
       'placeVisit.location.placeId', 'placeVisit.location.address',
       'placeVisit.location.semanticType',
       'placeVisit.location.locationConfidence',
       'placeVisit.duration.startTimestampMs',
       'placeVisit.duration.endTimestampMs', 'placeVisit.centerLatE7',
       'placeVisit.centerLngE7', 'placeVisit.visitConfidence',
       'placeVisit.otherCandidateLocations',
       'placeVisit.editConfirmationStatus', 'placeVisit.location.name',
       'placeVisit.simplifiedRawPath.points', 'placeVisit.childVisits',
       'placeVisit.location.sourceInfo.deviceTag',
       'placeVisit.placeConfidence'],
      dtype='object')

In [168]:
df = df[['placeVisit.location.latitudeE7', 'placeVisit.location.longitudeE7',
        'placeVisit.location.address',
       'placeVisit.duration.startTimestampMs',
       'placeVisit.duration.endTimestampMs', 'placeVisit.location.name'
]]

In [169]:
df.columns = [
    'latitudeE7', 'longitudeE7',
    'address', 'startTimestampMs',
    'endTimestampMs', 'name'
]

In [170]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1821 entries, 5 to 117
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   latitudeE7        1821 non-null   float64
 1   longitudeE7       1821 non-null   float64
 2   address           1804 non-null   object 
 3   startTimestampMs  1821 non-null   object 
 4   endTimestampMs    1821 non-null   object 
 5   name              1821 non-null   object 
dtypes: float64(2), object(4)
memory usage: 99.6+ KB


In [172]:
df['startTime'] = pd.to_datetime(df['startTimestampMs'], unit='ms')
df['endTime'] = pd.to_datetime(df['endTimestampMs'], unit='ms')

C:\Users\patry\Anaconda3\envs\aula\lib\site-packages\pandas\core\frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


In [173]:
df.head(3)

,latitudeE7,longitudeE7,address,startTimestampMs,endTimestampMs,name,startTime,endTime
5,-38651245.0,-386326169.0,"Av. Padre José Holanda do Vale, 2071 - Centro\...",1379959926740,1379963294968,CEJAM - Centro de Educação de Jovens e Adultos...,2013-09-23 18:12:06.740,2013-09-23 19:08:14.968
6,-38649812.0,-386328513.0,"Avenida Pe José Holanda do Vale, 2071 - Centro...",1379968186100,1379969773492,Conselho Escolar da Escola Municipal de Ensino...,2013-09-23 20:29:46.100,2013-09-23 20:56:13.492
0,-38354308.0,-385539145.0,Pref. José Walter\nFortaleza - CE\n60810-670\n...,1408143929408,1408146505596,Refla Construções e Serviços,2014-08-15 23:05:29.408,2014-08-15 23:48:25.596


In [174]:
df['time'] = df['endTime'] - df['startTime']

In [176]:
df.groupby('name').agg({'time': 'sum'}).sort_values(by='time', ascending=False).head(10)

,time
name,
IFCE - Federal Institute of Ceará Annex Aldeota,29 days 16:56:06.100000
Centro de Humanidades - Área III da Universidade Federal do Ceará,9 days 20:48:13.511000
Lava Jato Mota Júnior,8 days 23:03:44.223000
Universidade Federal do Ceará,8 days 00:19:18.053000
Salão De Festas Luiza Morais,7 days 17:45:40.277000
Shopping Benfica,7 days 09:01:50.001000
E.E.M Adauto Bezerra,5 days 18:37:22.165000
Ceará State University - Itaperi Campus,5 days 05:17:12.270000
Supermercado Frangolândia Itaperi,5 days 02:27:50.216000


The latitude and longitude columns came multiplied by 1e7, so I divided them to take the right values

In [178]:
df['latitude'] = df['latitudeE7'] / 1e7
df['longitude'] = df['longitudeE7'] / 1e7

In [185]:
df[[
    'name', 'address',
    'latitude', 'longitude',
    'startTime', 'endTime', 'time', 
]].to_csv('visited_places_history.csv', index=False)

In [181]:
df

,latitudeE7,longitudeE7,address,startTimestampMs,endTimestampMs,name,startTime,endTime,time,latitude,longitude
5,-38651245.0,-386326169.0,"Av. Padre José Holanda do Vale, 2071 - Centro\...",1379959926740,1379963294968,CEJAM - Centro de Educação de Jovens e Adultos...,2013-09-23 18:12:06.740,2013-09-23 19:08:14.968,0 days 00:56:08.228000,-3.865124,-38.632617
6,-38649812.0,-386328513.0,"Avenida Pe José Holanda do Vale, 2071 - Centro...",1379968186100,1379969773492,Conselho Escolar da Escola Municipal de Ensino...,2013-09-23 20:29:46.100,2013-09-23 20:56:13.492,0 days 00:26:27.392000,-3.864981,-38.632851
0,-38354308.0,-385539145.0,Pref. José Walter\nFortaleza - CE\n60810-670\n...,1408143929408,1408146505596,Refla Construções e Serviços,2014-08-15 23:05:29.408,2014-08-15 23:48:25.596,0 days 00:42:56.188000,-3.835431,-38.553914
8,-37759161.0,-385621512.0,"Rua Germano Franck, 300 - Parangaba\nFortaleza...",1408831660573,1408838400000,Casa dos Relojoeiros,2014-08-23 22:07:40.573,2014-08-24 00:00:00.000,0 days 01:52:19.427000,-3.775916,-38.562151
18,-38064380.0,-385804220.0,R. Prof. Glauco Lobo - Mondubim\nFortaleza - C...,1409058234741,1409064123570,CUCA Mondubim,2014-08-26 13:03:54.741,2014-08-26 14:42:03.570,0 days 01:38:08.829000,-3.806438,-38.580422
...,...,...,...,...,...,...,...,...,...,...,...
102,-37882720.0,-385533271.0,"Av. Dr. Silas Munguba, 1700 - Itaperi\nFortale...",1557450401529,1557450784459,Ceará State University - Itaperi Campus,2019-05-10 01:06:41.529,2019-05-10 01:13:04.459,0 days 00:06:22.930000,-3.788272,-38.553327
107,-37576232.0,-385308386.0,"762 telefone\nAv. Borges de Melo, 1630 - Fátim...",1557484933529,1557488710291,Terminal Rodoviário Eng. João Thomé,2019-05-10 10:42:13.529,2019-05-10 11:45:10.291,0 days 01:02:56.762000,-3.757623,-38.530839
111,-37289025.0,-385308744.0,"R. Pedro Pereira, 620 - Centro\nFortaleza - CE...",1557513009922,1557522173314,Dourados Musical,2019-05-10 18:30:09.922,2019-05-10 21:02:53.314,0 days 02:32:43.392000,-3.728902,-38.530874
114,-37414842.0,-385402656.0,"Av. da Universidade, 2853 - Benfica\nFortaleza...",1557523273237,1557525812571,Universidade Federal do Ceará,2019-05-10 21:21:13.237,2019-05-10 22:03:32.571,0 days 00:42:19.334000,-3.741484,-38.540266
